In [1]:
##### add python path #####
import sys
import os
PATH = os.getcwd()
for dir_idx, dir_name in enumerate(PATH.split('/')):
    if 'optimal_control' in dir_name.lower():
        PATH = '/'.join(PATH.split('/')[:(dir_idx+1)])
        break
if not PATH in sys.path:
    sys.path.append(PATH)
###########################
from env.quadruped.env import Env

from trajectory_planner import TrajectoryPlanner
from foot_step_planner import FootStepPlanner
from gait_scheduler import GaitScheduler
from NLP_solver import NLPSolver

from scipy.spatial.transform import Rotation
from copy import deepcopy
import numpy as np
import time

In [2]:
env = Env(enable_draw=True, base_fix=False)
num_leg = env.num_leg
time_step = env.time_step
base_inertia = deepcopy(env.model.inertia)
base_mass = env.model.mass
friction_coef = env.friction_coef
max_force = env.model.max_force

plan_time_step = 0.05
plan_time_horizon = 10

gs_args = dict()
gs_args['num_leg'] = num_leg
gs_args['gait_period'] = 1.0
gs_args['is_visualize'] = False
gait_scheduler = GaitScheduler(gs_args)

tp_args = dict()
tp_args['time_step'] = plan_time_step
tp_args['time_horizon'] = plan_time_horizon
tp_args['max_accel'] = 0.1
tp_args['max_ang_accel'] = 0.1
trajectory_planner = TrajectoryPlanner(tp_args)

fsp_args = dict()
fsp_args['num_leg'] = num_leg
fsp_args['time_horizon'] = plan_time_horizon
fsp_args['abduct_org_list'] = env.model.abduct_org_list
foot_step_planner = FootStepPlanner(fsp_args)

state_dim = 12
force_dim = 12
R_mat = np.eye(force_dim)*0.01 #001
Q_mat = np.eye(state_dim)
Qf_mat = np.eye(state_dim)*100.0
nlp_args = dict()
nlp_args['R_mat'] = R_mat
nlp_args['Q_mat'] = Q_mat
nlp_args['Qf_mat'] = Qf_mat
nlp_args['time_horizon'] = plan_time_horizon
nlp_args['time_step'] = plan_time_step
nlp_args['base_inertia'] = base_inertia
nlp_args['base_mass'] = base_mass
nlp_args['friction_coef'] = friction_coef
nlp_args['max_force'] = max_force
nlp_solver = NLPSolver(nlp_args)

In [3]:
env.reset()
nlp_solver.reset()
gait_scheduler.reset()
trajectory_planner.reset()
foot_step_planner.reset()

In [4]:
import imageio

class Renderer:
    def __init__(self, env, fps):
        self.env = env
        self.fps = fps
        self.RENDER_WIDTH, self.RENDER_HEIGHT = 800, 600

        self.view_matrix = env.pybullet_client.computeViewMatrixFromYawPitchRoll(
            cameraTargetPosition=env.camTargetPos,
            distance=env.camDist,
            yaw=env.camYaw,
            pitch=env.camPitch,
            roll=0,
            upAxisIndex=2)
        self.proj_matrix = env.pybullet_client.computeProjectionMatrixFOV(
            fov=60, aspect=float(self.RENDER_WIDTH)/self.RENDER_HEIGHT,
            nearVal=0.1, farVal=100.0)
        
        self.reset()
        
    
    def reset(self):
        self.global_t = 0.0
        self.img_list = []
    
    def render(self, time_step):
        self.global_t += time_step
        while self.global_t*self.fps > len(self.img_list):
            width, height, rgb_img, depth_img, seg_img = self.env.pybullet_client.getCameraImage(
                width=self.RENDER_WIDTH, 
                height=self.RENDER_HEIGHT,
                viewMatrix=self.view_matrix,
                projectionMatrix=self.proj_matrix)
            rgb_img = rgb_img[:,:,:3]
            bgr_img = rgb_img[:,:,::-1]
            self.img_list.append(rgb_img)

    def save_video(self):
        output = "./result.mp4"
        writer = imageio.get_writer(output, fps=self.fps)
        for i, img in enumerate(self.img_list):
            sys.stdout.write("\rframe {0}".format(i))
            sys.stdout.flush()
            writer.append_data(img)
        print("\r\nFinalizing...")
        writer.close()
        
renderer = Renderer(env, fps=30)

In [5]:
'''
gait_list = np.zeros((num_leg, 2)) # FR, FL, RR, RL
gait_list[0,0] = 0.5 # offset
gait_list[0,1] = 0.5 # duration
gait_list[2,0] = 0.0 # offset
gait_list[2,1] = 0.5 # duration
gait_scheduler.gait_list = deepcopy(gait_list)
gait_scheduler.update()
'''

vel_cmd = np.zeros(3)
vel_cmd[0] = 0.0
vel_cmd[1] = 0.0
ang_vel_cmd = np.zeros(3)
ang_vel_cmd[2] = 0.0
height_cmd = 0.3

In [6]:
'''
cube = env.pybullet_client.loadURDF("cube_small.urdf")

collisionFilterGroup = 0x1
collisionFilterMask = 0x1
env.pybullet_client.setCollisionFilterGroupMask(env.planeId, -1, collisionFilterGroup, collisionFilterMask)
for i in range(env.pybullet_client.getNumJoints(env.model.sim_model)):
    env.pybullet_client.setCollisionFilterGroupMask(env.model.sim_model, i, collisionFilterGroup, collisionFilterMask)

collisionFilterGroup2 = 0x2
collisionFilterMask2 = 0x2 
for i in range(env.pybullet_client.getNumJoints(cube)):
    env.pybullet_client.setCollisionFilterGroupMask(cube, i, collisionFilterGroup2, collisionFilterMask2)
'''

'\ncube = env.pybullet_client.loadURDF("cube_small.urdf")\n\ncollisionFilterGroup = 0x1\ncollisionFilterMask = 0x1\nenv.pybullet_client.setCollisionFilterGroupMask(env.planeId, -1, collisionFilterGroup, collisionFilterMask)\nfor i in range(env.pybullet_client.getNumJoints(env.model.sim_model)):\n    env.pybullet_client.setCollisionFilterGroupMask(env.model.sim_model, i, collisionFilterGroup, collisionFilterMask)\n\ncollisionFilterGroup2 = 0x2\ncollisionFilterMask2 = 0x2 \nfor i in range(env.pybullet_client.getNumJoints(cube)):\n    env.pybullet_client.setCollisionFilterGroupMask(cube, i, collisionFilterGroup2, collisionFilterMask2)\n'

In [7]:
global_t = 0.0
mpc_cnt = 0
mpc_period = int(plan_time_step/time_step)
while global_t < 10.0:
    if mpc_cnt == 0:
        init_state_list = np.concatenate([env.model.com_pos, env.model.base_rpy, env.model.base_vel, env.model.base_ang_vel])
        init_foot_pos_list = deepcopy(env.model.foot_pos_list)
        
        #target_trajectory_list = trajectory_planner.get_target_trajectory_list(vel_cmd, ang_vel_cmd, height_cmd, init_state)
        target_trajectory_list = np.zeros((plan_time_horizon, 12))
        target_trajectory_list[:,2] = height_cmd
        contact_list, step_info_list = gait_scheduler.get_contact_list(plan_time_step, plan_time_horizon)
        target_foot_step_list = foot_step_planner.get_target_foot_step_list(init_foot_pos_list, step_info_list, target_trajectory_list, vel_cmd, ang_vel_cmd, height_cmd)
        state_list, force_list = nlp_solver.get_solution(init_state_list, target_trajectory_list, target_foot_step_list, contact_list)

        '''
        for i in range(plan_time_horizon):
            time.sleep(0.1)
            state = state_list[i]
            pos_base = state[:3]
            rpy_base = state[3:6]
            vel_base = state[6:9]
            ang_vel_base = state[9:]
            orn_base = env.pybullet_client.getQuaternionFromEuler(rpy_base)

            env.pybullet_client.resetBasePositionAndOrientation(cube, pos_base, orn_base)
            env.pybullet_client.resetBaseVelocity(cube, np.zeros(3), np.zeros(3))
        '''
        print(state_list[:,:3])

    contact_list, gait_info_list = gait_scheduler.step(time_step)
    inter_weight = 1.0 - float(mpc_cnt)/mpc_period
    inter_force_list = inter_weight*force_list[0] + (1-inter_weight)*force_list[1]
    env.step(inter_force_list, contact_list, desired_foot_list=[])
    renderer.render(time_step)
    #time.sleep(time_step)

    mpc_cnt = np.mod(mpc_cnt + 1, mpc_period)
    global_t += time_step
    
renderer.save_video()
gait_scheduler.close()

[[-5.10818839e-03  7.14753824e-07  1.72948376e-01]
 [-4.68640974e-03  1.34130677e-06  1.52084925e-01]
 [-4.02030554e-03  7.99154417e-07  1.44773115e-01]
 [-3.34002202e-03 -1.56136950e-06  1.48213468e-01]
 [-2.78785628e-03 -6.68238382e-06  1.59613460e-01]
 [-2.49602333e-03 -1.57752050e-05  1.76186524e-01]
 [-2.59919859e-03 -3.02828855e-05  1.95152133e-01]
 [-3.19548751e-03 -5.17947667e-05  2.13737317e-01]
 [-4.33141806e-03 -8.16432012e-05  2.29178027e-01]
 [-6.02574010e-03 -1.18223177e-04  2.38716407e-01]]
[[ 3.10138721e-03  3.06836003e-06  1.75207860e-01]
 [ 6.47284500e-03  3.37570875e-06  1.75176264e-01]
 [ 9.21553997e-03  3.22786792e-06  1.85504537e-01]
 [ 1.14488357e-02  2.06374228e-06  2.03458204e-01]
 [ 1.32447462e-02 -8.66297684e-07  2.26311085e-01]
 [ 1.46377640e-02 -6.42232228e-06  2.51349799e-01]
 [ 1.56676214e-02 -1.56513751e-05  2.75877252e-01]
 [ 1.63787167e-02 -2.93692425e-05  2.97212899e-01]
 [ 1.67858135e-02 -4.52275060e-05  3.12684953e-01]
 [ 1.67430202e-02 -4.80139982e

[[-0.05270718 -0.00032405  0.26837196]
 [-0.05375202 -0.00034645  0.27051591]
 [-0.05409886 -0.00035779  0.27288448]
 [-0.05401456 -0.00035988  0.27527305]
 [-0.05369194 -0.00035383  0.27752441]
 [-0.05326078 -0.00034152  0.2795162 ]
 [-0.0528029  -0.00032532  0.28115634]
 [-0.05236479 -0.00030833  0.28237445]
 [-0.05196877 -0.00029403  0.28311458]
 [-0.05162365 -0.00028574  0.28332746]]
[[-5.44990966e-02 -3.56567263e-04  2.69657730e-01]
 [-5.54376243e-02 -3.75670312e-04  2.71534977e-01]
 [-5.57258288e-02 -3.82382672e-04  2.73636168e-01]
 [-5.56155543e-02 -3.77980589e-04  2.75771507e-01]
 [-5.52821627e-02 -3.64544579e-04  2.77791503e-01]
 [-5.48431601e-02 -3.44588855e-04  2.79580616e-01]
 [-5.43732240e-02 -3.21280362e-04  2.81049983e-01]
 [-5.39151886e-02 -2.98242429e-04  2.82132914e-01]
 [-5.34887360e-02 -2.79399717e-04  2.82777459e-01]
 [-5.31008064e-02 -2.68594693e-04  2.82936325e-01]]
[[-5.61351492e-02 -3.86372271e-04  2.70811844e-01]
 [-5.69715975e-02 -4.01341403e-04  2.72517738e-

[[-6.88283343e-02 -3.88476056e-04  2.79691057e-01]
 [-6.86165015e-02 -3.63724615e-04  2.79980928e-01]
 [-6.80470742e-02 -3.31730220e-04  2.80144819e-01]
 [-6.72677475e-02 -3.01446233e-04  2.80099915e-01]
 [-6.63869541e-02 -2.79531774e-04  2.79834919e-01]
 [-6.54826840e-02 -2.66957184e-04  2.79369261e-01]
 [-6.46100953e-02 -2.59983727e-04  2.78783099e-01]
 [-6.38013385e-02 -2.51030024e-04  2.78171794e-01]
 [-6.30672613e-02 -2.36383821e-04  2.77613974e-01]
 [-6.24019149e-02 -2.26436790e-04  2.77167800e-01]]
[[-6.91022661e-02 -3.77258051e-04  2.79929305e-01]
 [-6.88516866e-02 -3.55787825e-04  2.80196747e-01]
 [-6.82424909e-02 -3.31679579e-04  2.80309123e-01]
 [-6.74254067e-02 -3.11704872e-04  2.80216319e-01]
 [-6.65102934e-02 -2.97144979e-04  2.79905358e-01]
 [-6.55780783e-02 -2.84537850e-04  2.79428410e-01]
 [-6.46831541e-02 -2.66589234e-04  2.78863617e-01]
 [-6.38557258e-02 -2.39977914e-04  2.78292172e-01]
 [-6.31035646e-02 -2.15973207e-04  2.77798693e-01]
 [-6.24180436e-02 -2.00576868e

[[-6.94305410e-02 -1.76926780e-04  2.81526230e-01]
 [-6.87411979e-02 -1.55399989e-04  2.81274531e-01]
 [-6.77754549e-02 -1.34517888e-04  2.80724272e-01]
 [-6.66700973e-02 -1.19744966e-04  2.79875975e-01]
 [-6.55224130e-02 -1.13288756e-04  2.78755622e-01]
 [-6.44018216e-02 -1.12255952e-04  2.77416463e-01]
 [-6.33538572e-02 -1.12175719e-04  2.76001132e-01]
 [-6.24090666e-02 -1.09495064e-04  2.74696933e-01]
 [-6.15789426e-02 -1.02549952e-04  2.73674266e-01]
 [-6.08565451e-02 -9.51203414e-05  2.72996033e-01]]
[[-6.91848491e-02 -1.69331426e-04  2.81483804e-01]
 [-6.85026021e-02 -1.56229932e-04  2.81219960e-01]
 [-6.75538957e-02 -1.45982849e-04  2.80636132e-01]
 [-6.64722929e-02 -1.41046378e-04  2.79714928e-01]
 [-6.53525760e-02 -1.38825761e-04  2.78471625e-01]
 [-6.42599708e-02 -1.35063277e-04  2.77015624e-01]
 [-6.32422760e-02 -1.26358201e-04  2.75516314e-01]
 [-6.23269802e-02 -1.11311926e-04  2.74157161e-01]
 [-6.15176625e-02 -9.44589465e-05  2.73056854e-01]
 [-6.08060897e-02 -8.24040634e

[[-0.06717094 -0.00035126  0.28184388]
 [-0.0667481  -0.00038732  0.2821809 ]
 [-0.06602278 -0.00041121  0.28264947]
 [-0.06514911 -0.00042661  0.28321057]
 [-0.06423278 -0.00042942  0.28378805]
 [-0.06334676 -0.00042195  0.28431379]
 [-0.0625342  -0.00040789  0.28473114]
 [-0.06181657 -0.00039161  0.28499445]
 [-0.0612003  -0.00037796  0.28506675]
 [-0.06068651 -0.00037192  0.28491213]]
[[-0.06740904 -0.00039278  0.28207198]
 [-0.06706977 -0.00042097  0.28255019]
 [-0.06641342 -0.00043833  0.28319684]
 [-0.06559356 -0.00044043  0.28391107]
 [-0.06471985 -0.00042948  0.28460808]
 [-0.06386518 -0.00040905  0.28521871]
 [-0.06307577 -0.00038343  0.28568923]
 [-0.06237653 -0.00035757  0.28598098]
 [-0.06177699 -0.0003373   0.28606248]
 [-0.06128053 -0.0003285   0.28590267]]
[[-6.77535153e-02 -4.27378400e-04  2.82356643e-01]
 [-6.74983434e-02 -4.50320590e-04  2.82930220e-01]
 [-6.69070908e-02 -4.54836493e-04  2.83653264e-01]
 [-6.61373743e-02 -4.42805761e-04  2.84417053e-01]
 [-6.52988772e

...

[[-6.25165895e-02 -1.31238752e-04  2.87288379e-01]
 [-6.19457210e-02 -1.13533938e-04  2.87079221e-01]
 [-6.11101126e-02 -9.63175820e-05  2.86561225e-01]
 [-6.01375895e-02 -8.30555733e-05  2.85749403e-01]
 [-5.91168603e-02 -7.62709890e-05  2.84692415e-01]
 [-5.81097929e-02 -7.53200770e-05  2.83466137e-01]
 [-5.71576109e-02 -7.65223241e-05  2.82169979e-01]
 [-5.62865163e-02 -7.64667462e-05  2.80949894e-01]
 [-5.55074979e-02 -7.44967442e-05  2.79941260e-01]
 [-5.48269097e-02 -7.24117400e-05  2.79232660e-01]]
[[-6.23428685e-02 -1.19691048e-04  2.87293803e-01]
 [-6.17396214e-02 -1.05587325e-04  2.87062506e-01]
 [-6.08733133e-02 -9.29728778e-05  2.86512415e-01]
 [-5.98705503e-02 -8.44091339e-05  2.85654475e-01]
 [-5.88185373e-02 -7.92986243e-05  2.84532157e-01]
 [-5.77790658e-02 -7.40038466e-05  2.83218150e-01]
 [-5.67972144e-02 -6.51802385e-05  2.81845762e-01]
 [-5.58996075e-02 -5.23848552e-05  2.80566172e-01]
 [-5.50980803e-02 -3.84228959e-05  2.79522060e-01]
 [-5.43981574e-02 -2.77483550e

[[-5.96603479e-02 -1.85072899e-04  2.87418490e-01]
 [-5.91986590e-02 -1.96080770e-04  2.87532314e-01]
 [-5.84649526e-02 -2.02438398e-04  2.87579317e-01]
 [-5.75934823e-02 -2.04824878e-04  2.87527755e-01]
 [-5.66770440e-02 -1.99923540e-04  2.87348361e-01]
 [-5.57800737e-02 -1.88392932e-04  2.87039392e-01]
 [-5.49442941e-02 -1.72385579e-04  2.86624745e-01]
 [-5.41911862e-02 -1.54749345e-04  2.86146360e-01]
 [-5.35287153e-02 -1.39123464e-04  2.85647476e-01]
 [-5.29644331e-02 -1.29399370e-04  2.85147603e-01]]
[[-5.97423750e-02 -1.94296167e-04  2.87559931e-01]
 [-5.93206942e-02 -2.01316424e-04  2.87699016e-01]
 [-5.86197367e-02 -2.02864162e-04  2.87751077e-01]
 [-5.77746268e-02 -1.95517529e-04  2.87661316e-01]
 [-5.68792868e-02 -1.79859681e-04  2.87407689e-01]
 [-5.59993584e-02 -1.57919191e-04  2.86998160e-01]
 [-5.51771714e-02 -1.32434106e-04  2.86468917e-01]
 [-5.44355649e-02 -1.07119795e-04  2.85876963e-01]
 [-5.37843870e-02 -8.66652584e-05  2.85281445e-01]
 [-5.32333928e-02 -7.62647339e

[[-5.94637218e-02 -1.52689948e-04  2.88907432e-01]
 [-5.89446459e-02 -1.73048780e-04  2.89012475e-01]
 [-5.81494252e-02 -1.92858663e-04  2.89062045e-01]
 [-5.72166557e-02 -2.12591066e-04  2.89051471e-01]
 [-5.62405280e-02 -2.26100466e-04  2.88957645e-01]
 [-5.52864923e-02 -2.32069443e-04  2.88775106e-01]
 [-5.43979311e-02 -2.31710396e-04  2.88511497e-01]
 [-5.35972813e-02 -2.27261078e-04  2.88185761e-01]
 [-5.28932697e-02 -2.21576633e-04  2.87818294e-01]
 [-5.22957936e-02 -2.17532653e-04  2.87414466e-01]]
[[-5.94949458e-02 -1.70332571e-04  2.89019829e-01]
 [-5.90155699e-02 -1.88564709e-04  2.89149908e-01]
 [-5.82567153e-02 -2.06307388e-04  2.89239039e-01]
 [-5.73559860e-02 -2.17163543e-04  2.89241125e-01]
 [-5.64085859e-02 -2.19653655e-04  2.89134010e-01]
 [-5.54817314e-02 -2.14822003e-04  2.88912931e-01]
 [-5.46185020e-02 -2.04789079e-04  2.88592181e-01]
 [-5.38418239e-02 -1.92445999e-04  2.88201434e-01]
 [-5.31611845e-02 -1.81197017e-04  2.87772645e-01]
 [-5.25868406e-02 -1.74204105e

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


frame 300
Finalizing...


In [9]:
renderer.save_video()

frame 0

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


frame 300
Finalizing...


In [11]:
def get_cross_product_matrix(vector):
    a, b, c = vector
    mat = np.array([[0, -c, b],
                   [c, 0, -a],
                   [-b, a, 0]])
    return mat

def eq_cons_func(x, args):
    init_state_list = args[0]
    target_foot_step_list = args[1]
    contact_list = args[2]
    delta_t = args[3]
    base_inertia = args[4]
    base_mass = args[5]
    num_leg = 4

    time_horizon = int(len(x)/24)
    state_list = x[:int(len(x)/2)].reshape((time_horizon, 12))
    force_list = x[int(len(x)/2):].reshape((time_horizon, 12))
    
    eq_cons = []
    state = init_state_list
    gravity = np.zeros(12)
    gravity[6:9] = delta_t*np.array([0, 0, -9.8])
    for time_idx in range(time_horizon):
        next_state = state_list[time_idx]
        force = (force_list[time_idx].reshape((4,3))*contact_list[time_idx].reshape(4,1)).ravel()

        yaw = state[5]
        pos_com = state[:3]
        yaw_mat = Rotation.from_euler('z', yaw, degrees=False).as_matrix()
        global_inertia = np.matmul(yaw_mat, np.matmul(base_inertia, yaw_mat.T))
        inv_global_inertia = np.linalg.inv(global_inertia)
        foot_step_list = target_foot_step_list[time_idx]

        A_mat = np.eye(12)
        A_mat[0:3, 6:9] = delta_t*np.eye(3)
        A_mat[3:6, 9:12] = delta_t*yaw_mat.T #delta_t*yaw_mat

        B_mat = np.zeros((12, 12))
        for leg_idx in range(num_leg):
            foot_step = foot_step_list[leg_idx] - pos_com
            B_mat[6:9, leg_idx*3:(leg_idx+1)*3] = np.eye(3)*(delta_t/base_mass)
            B_mat[9:12, leg_idx*3:(leg_idx+1)*3] = delta_t*np.matmul(inv_global_inertia, get_cross_product_matrix(foot_step))
        
        eq_cons.append((np.matmul(A_mat, state) + np.matmul(B_mat, force) + gravity - next_state).ravel())
        state = next_state

    eq_cons = np.concatenate(eq_cons)
    return eq_cons

def eq_cons_jacobian(x, args):
    init_state_list = args[0]
    target_foot_step_list = args[1]
    contact_list = args[2]
    delta_t = args[3]
    base_inertia = args[4]
    base_mass = args[5]
    num_leg = 4

    time_horizon = int(len(x)/24)
    state_list = x[:int(len(x)/2)].reshape((time_horizon, 12))
    force_list = x[int(len(x)/2):].reshape((time_horizon, 12))
    
    jacobian = np.zeros((12*time_horizon, len(x)))
    state = init_state_list
    for time_idx in range(time_horizon):        
        next_state = state_list[time_idx]
        force = (force_list[time_idx].reshape((4,3))*contact_list[time_idx].reshape(4,1)).ravel()

        yaw = state[5]
        pos_com = state[:3]
        ang_vel_base = state[9:12]
        yaw_mat = Rotation.from_euler('z', yaw, degrees=False).as_matrix()
        global_inertia = np.matmul(yaw_mat, np.matmul(base_inertia, yaw_mat.T))
        inv_global_inertia = np.linalg.inv(global_inertia)
        inv_base_inertia = np.linalg.inv(base_inertia)
        foot_step_list = target_foot_step_list[time_idx]

        A_mat = np.eye(12)
        A_mat[0:3, 6:9] = delta_t*np.eye(3)
        A_mat[3:6, 9:12] = delta_t*yaw_mat.T #delta_t*yaw_mat

        B_mat = np.zeros((12, 12))
        for leg_idx in range(num_leg):
            foot_step = foot_step_list[leg_idx] - pos_com
            B_mat[6:9, leg_idx*3:(leg_idx+1)*3] = np.eye(3)*(delta_t/base_mass)
            B_mat[9:12, leg_idx*3:(leg_idx+1)*3] = delta_t*np.matmul(inv_global_inertia, get_cross_product_matrix(foot_step))
        
        jacobian[time_idx*12:(time_idx+1)*12, time_idx*12:(time_idx+1)*12] = -np.eye(12)
        c1, c2, c3, c4 = contact_list[time_idx]
        contact_mat = np.diag([c1]*3 + [c2]*3 + [c3]*3 + [c4]*3)
        jacobian[time_idx*12:(time_idx+1)*12, (time_horizon + time_idx)*12:(time_horizon + time_idx + 1)*12] = np.matmul(B_mat, contact_mat)
        if time_idx > 0:
            jacobian[time_idx*12:(time_idx+1)*12, (time_idx-1)*12:time_idx*12] = A_mat

            temp_mat = np.zeros((12, 12))
            derivative_R = np.array([[-np.sin(yaw), -np.cos(yaw), 0],[np.cos(yaw), -np.sin(yaw), 0],[0, 0, 0]])
            temp_mat[3:6, 5] += delta_t*np.matmul(derivative_R.T, ang_vel_base)
            sum_r_cross_f = np.zeros(3)
            for leg_idx in range(num_leg):
                foot_step = foot_step_list[leg_idx] - pos_com
                sum_r_cross_f += delta_t*np.cross(foot_step, force[leg_idx*3:(leg_idx+1)*3])
            temp_mat[9:12, 5] += np.matmul(derivative_R, np.matmul(inv_base_inertia, np.matmul(yaw_mat.T, sum_r_cross_f)))
            temp_mat[9:12, 5] += np.matmul(yaw_mat, np.matmul(inv_base_inertia, np.matmul(derivative_R.T, sum_r_cross_f)))
            jacobian[time_idx*12:(time_idx+1)*12, (time_idx-1)*12:time_idx*12] += temp_mat

        state = next_state
    return jacobian

In [47]:
init_state_list = np.concatenate([env.model.com_pos, env.model.base_rpy, env.model.base_vel, env.model.base_ang_vel])
init_foot_pos_list = deepcopy(env.model.foot_pos_list)

target_trajectory_list = trajectory_planner.get_target_trajectory_list(vel_cmd, ang_vel_cmd, height_cmd, init_state_list)
contact_list, step_info_list = gait_scheduler.get_contact_list(plan_time_step, plan_time_horizon)
target_foot_step_list = foot_step_planner.get_target_foot_step_list(init_foot_pos_list, step_info_list, target_trajectory_list, vel_cmd, ang_vel_cmd, height_cmd)

In [48]:
args = [init_state_list, target_foot_step_list, contact_list, plan_time_step, base_inertia, base_mass]

init_force_list = np.zeros((plan_time_horizon, 12))
init_x_list = np.concatenate([target_trajectory_list.ravel(), init_force_list.ravel()])

In [49]:
approx_jacobian = np.zeros((12*plan_time_horizon, 24*plan_time_horizon))
eps = 1e-10
func_value = eq_cons_func(init_x_list , args)
for idx in range(len(init_x_list)):
    temp_x_list = np.zeros_like(init_x_list)
    temp_x_list[idx] = eps
    approx_jacobian[:, idx] = (eq_cons_func(init_x_list + temp_x_list, args) - func_value)/eps

In [50]:
jacobian = eq_cons_jacobian(init_x_list, args)

In [51]:
np.sum(abs(jacobian - approx_jacobian) > 1e-5)

0

In [46]:
cnt = 0
mpc_cnt = 0
mpc_period = int(plan_time_step/time_step)
while cnt < mpc_period:
    if mpc_cnt == 0:
        init_state_list = np.concatenate([env.model.com_pos, env.model.base_rpy, env.model.base_vel, env.model.base_ang_vel])
        init_foot_pos_list = deepcopy(env.model.foot_pos_list)

        target_trajectory_list = trajectory_planner.get_target_trajectory_list(vel_cmd, ang_vel_cmd, height_cmd, init_state_list)
        contact_list, step_info_list = gait_scheduler.get_contact_list(plan_time_step, plan_time_horizon)
        target_foot_step_list = foot_step_planner.get_target_foot_step_list(init_foot_pos_list, step_info_list, target_trajectory_list, vel_cmd, ang_vel_cmd, height_cmd)
        state_list, force_list = nlp_solver.get_solution(init_state_list, target_trajectory_list, target_foot_step_list, contact_list)

        '''
        for i in range(plan_time_horizon):
            time.sleep(0.1)
            state = state_list[i]
            pos_base = state[:3]
            rpy_base = state[3:6]
            vel_base = state[6:9]
            ang_vel_base = state[9:]
            orn_base = env.pybullet_client.getQuaternionFromEuler(rpy_base)

            env.pybullet_client.resetBasePositionAndOrientation(cube, pos_base, orn_base)
            env.pybullet_client.resetBaseVelocity(cube, np.zeros(3), np.zeros(3))
            #print(pos_base, target_trajectory_list[i,:3])
        '''

    contact_list, gait_info_list = gait_scheduler.step(time_step)
    inter_weight = 1.0 - float(mpc_cnt)/mpc_period
    inter_force_list = inter_weight*force_list[0] + (1-inter_weight)*force_list[1]
    #print(inter_force_list)
    env.step(inter_force_list, contact_list, desired_foot_list=[])
    time.sleep(time_step)

    mpc_cnt = np.mod(mpc_cnt + 1, mpc_period)
    cnt += 1

Iteration limit exceeded    (Exit mode 9)
            Current function value: 34.9787148966616
            Iterations: 51
            Function evaluations: 488
            Gradient evaluations: 51
